In [1]:
!pip install -q dotenv langchain-google-genai ollama pandas

In [2]:
import os

os.environ["OLLAMA_HOST"] = "https://a226-35-240-130-207.ngrok-free.app"

In [3]:
!ollama list

NAME           ID              SIZE      MODIFIED      
llama3.2:3b    a80c4f17acd5    2.0 GB    4 minutes ago    
gemma3:4b      c0494fe00251    3.3 GB    5 minutes ago    


In [4]:
!ollama pull gemma3:4b
!ollama pull llama3.2:3b
!ollama list

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 377655e65351... 100% ▕████████████████▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕████████████████▏  358 B                         
pulling dd084c7d92a3... 100% ▕████████████████▏ 8.4 KB                         
pulling 0a74a8735bf3... 100% ▕████████████████▏   55 B                         
pulling ffae984acbea... 100% ▕████████████████▏  489 B                         pulling manifest 
pulling 377655e65351... 100% ▕████████████████▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕████████████████▏  358 B                         
pulling dd084c7d92a3... 100% ▕████████████████▏ 8.4 KB                         
pulling 0a74a8735bf3.

NAME           ID              SIZE      MODIFIED                    
llama3.2:3b    a80c4f17acd5    2.0 GB    Less than a second from now    
gemma3:4b      c0494fe00251    3.3 GB    1 second ago                   


In [5]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

llm_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = os.getenv("GEMINI_API_KEY")
)


In [ ]:
# questions = [
#     "List all drugs in the 'Painkillers' category with a stock below 100 units, along with their supplier names and the date when their inventory was last updated. Order results by oldest inventory update first.",
#     "Identify the top 5 products by total sales amount in the last quarter of 2024, including their category names and the store locations where they were sold. Order the results by total sales amount in descending order.",
#     "Find the top 3 products with the highest average rating from user reviews, including their category names and total sales amounts. Only consider products that have been ordered at least 10 times. Order the results by average rating in descending order.",
#     "List all agents who have facilitated sales of properties with a total sale price exceeding $500,000 in the last year. Include their full names, the number of properties sold, and the total commission earned. Order the results by total commission in descending order.",
#     "Identify the top 5 products by total sales amount in the last quarter of 2024, including the name of the employee who made the sale and the total quantity sold for each product. Order the results by total sales amount in descending order.",
#     "Retrieve the first name of customers who have given a 5-star rating to a restaurant and have also placed at least one order at that restaurant. For each such customer-restaurant pair, also retrieve the name of the restaurant and the total amount the customer has spent at that restaurant.",
#     "Retrieve the first and last name of customers who have purchased a vehicle and also requested a service for that same vehicle, along with the model of the vehicle and the total cost of all services for that vehicle.",
#     "How can you retrieve a list of all active subscriptions along with the customer's name, plan name, and the subscription's start and end dates",
#     "How can you retrieve a report showing the total stock level of each product across all warehouses, including the product name and total quantity available?",
#     "How can you generate a report showing the total sales revenue for each salesperson, including their name and the total amount sold?",
# ]

# responses = []
# ratings = []

# import ollama

# # model_list = ['qwen2.5:7b','gemma3:4b','phi3','phi3.5']
# model_list = ['llama3.2:3b']

# dict=[]

# for model in model_list:
    
#     for i in range(10):
#         FILENAME = f"table_list{i+1}_exp.sql"

#         response = ollama.chat(model=model, messages=[
#         {
#             'role': 'user',
#             # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
#             'content': get_custom_prompt(
#                 question = questions[i],
#                 sql_file = FILENAME
#             )
#         },
#         ])

#         res = response['message']['content']
        

#         responses.append(res)

#         rating = get_ratings(
#             schema = f"table_list{i+1}_exp.sql",
#             question = questions[i],
#             answer = res,
#         )

#         ratings.append(rating)

#         d={
#             'schema':read_file(FILENAME),
#             'model':model,
#             'question':questions[i],
#             'sqlanswer':res,
#             'rating':rating
#         }

#         dict.append(d)





In [ ]:
def read_file(filename):
    file = open(filename,'r')
    str = file.read()
    return str


In [ ]:
# Prompt to generate query-Question from Schema
def get_question_prompt(sql_file,difficulty):  
    prompt = f"""  
    ### Instructions:  
    Your task is to thoroughly analyze the given database schema and generate a **SQL-RELATED QUESTION with DIFFICULTY LEVEL, {difficulty} **.  

    **Guidelines:**  
    - **DO NOT provide an SQL query.**  
    - **Carefully review every detail of the schema** before formulating the question.  
    - **Ensure the question requires the use of JOINS and ALIASES** to test complex query-building skills.  
    - The question **must be STRICTLY BASED ON GIVEN SCHEMA** — avoid assumptions beyond the provided structure.  
    - **Output ONLY the QUESTION.** No Additional Text. No introductory or concluding text.  

    ### Input:  
    Generate a well-structured question using the following database schema:  
    `{read_file(sql_file)}`  

    ### Expected Output:  
    A single, well-formed SQL-related question based strictly on the given schema.  

    """  
    return prompt

# Prompt to generate SQL-Query from query_Question & Schema
def get_custom_prompt(question, sql_file):  
    prompt = f"""  
    ### Instructions:  
    Your task is to **convert the given question into an accurate SQL query**, using the provided **Database schema**.  

    **Guidelines:**  
    - **Carefully analyze the question and schema word by word** before constructing the query.  
    - **Use appropriate JOINs and table aliases** to maintain CLARITY and prevent ambiguity.  
    - Always use **table aliases** when referencing columns from multiple tables.  
    - When computing a **ratio**, always cast the numerator as `FLOAT` to ensure precision.  

    ### Input:  
    Generate an SQL query to answer the following question:  
    `{question}`  

    This query will be executed on a database with the following schema:  
    `{read_file(sql_file)}`  

    ### Expected Output:  
    A **fully-formed SQL query** that precisely answers the question **without any additional text** or explanations.  

    """  
    return prompt  

# Prompt to Rate SQL-Query
def get_rate_prompt(schema, question, answer):  
    prompt = f"""  

    ### Instructions:  
    Your task is to **evaluate the given SQL query on a scale of 1 to 10**, based on its **efficiency and simplicity**.  

    **Guidelines:**  
    - **Thoroughly review the database schema and the given question** before rating.  
    - **Assess the SQL query for correctness, efficiency, and clarity** in answering the question.  
    - **Use table aliases** where necessary to maintain clarity and prevent ambiguity.  
    - When handling **ratios**, ensure that the numerator is cast as `FLOAT` for precision.  

    ### Input:  
    - **Question:** `{question}`  
    - **SQL Query:** `{answer}`  
    - **Database Schema:**  
    `{read_file(schema)}`  

    **IMPORTANT:**  
    - **Provide ONLY the rating (1-10) as output.**  
    - **No explanations. No additional text.**  

    ### Expected Output:  
    A single numeric rating from 1 to 10, without any extra text.  

    """  
    return prompt  




In [35]:
import ollama

# Function Rate SQLQuery
def get_ratings(schema,question,answer):
    response = llm_gemini.invoke(
        get_rate_prompt(
            schema = schema,
            question = question,
            answer = answer,
            )
    )
    
    return response.content

# Function to generate SQL-Query & Rate it
def generate_sqlquery_and_rate(schema_file,model,question):
    FILENAME = schema_file
    response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            'content': get_custom_prompt(
                question = question,
                sql_file = FILENAME
            )
        },
    ])
    res = response['message']['content']

    rating = get_ratings(
            schema = f"table_list{i+1}_exp.sql",
            question = question,
            answer = res,
        )
    # rating = rating['meassage']['content']

    d={
        'schema':read_file(FILENAME),
        'model':model,
        'question':question,
        'sqlanswer':res,
        'rating':rating
    }
    return d

# Function to generate Questions based on Difficulty
def generate_questions(schema_file,num_of_questions,difficulty_level):
    questions=[]
    for i in range(num_of_questions):
        llm_question = ollama.chat(model='gemma3:4b',messages=[
            {
                'role':'user',
                'content':get_question_prompt(
                    sql_file=schema_file,
                    difficulty=difficulty_level
                )
            }
        ])
        llm_question = llm_question['message']['content']
        questions.append(llm_question)
    return questions
    

In [28]:
easy_query_questions = []
for i in range(10):
    FILENAME = f"table_list{i+1}_exp.sql"
    ques=generate_questions(FILENAME,1,"EASY")
    easy_query_questions.append(ques)


In [ ]:
hard_query_questions = []
for i in range(10):
    FILENAME = f"table_list{i+1}_exp.sql"
    ques=generate_questions(FILENAME,1,"HARD")
    easy_query_questions.append(ques)


In [45]:
model_list = ['llama3.2:3b','gemma3:4b']

final_dict = []
for model in model_list:
    for i in range(len(easy_query_questions)):
        FILENAME = f"table_list{i+1}_exp.sql"

        d = generate_sqlquery_and_rate(
            schema_file=FILENAME,
            model=model,
            question=easy_query_questions[i]
        )
        
        final_dict.append(d)

        

In [47]:
import pandas as pd

df=pd.DataFrame(final_dict)
df.to_csv('Result_4_LLM_Generated_Easy_Questions.csv')
df

,schema,model,question,sqlanswer,rating
0,\n CREATE TABLE supplier (\n supplie...,llama3.2:3b,[Write a SQL query to identify all pharmacies ...,"SELECT T1.pharmacy_name , AVG(T2.rating) FROM...",7
1,CREATE TABLE store (\n store_id INT PRIMARY...,llama3.2:3b,[ Generate a SQL query to determine the tota...,"SELECT T1.store_name, SUM(T3.subtotal) / COUNT...",6
2,CREATE TABLE user (\n user_id INT PRIMARY K...,llama3.2:3b,[ \n Generate a SQL query to determine the ...,"SELECT \n c.category_name,\n AVG(ti.subt...",9
3,CREATE TABLE agent (\n agent_id INT PRIMARY...,llama3.2:3b,[ \n Generate a SQL query to determine the ...,"SELECT \n a.first_name,\n a.last_name,\n AV...",7
4,CREATE TABLE supplier (\n supplier_id INT P...,llama3.2:3b,[Generate a report showing the total sales rev...,"SELECT \n p.product_name, \n s.sale_price, \...",6
5,CREATE TABLE restaurant (\n restaurant_id I...,llama3.2:3b,[Write a SQL query to identify all restaurants...,"SELECT r.name, AVG(f.rating) AS avg_rating \nF...",2
6,CREATE TABLE vehicle_type (\n type_id INT P...,llama3.2:3b,[ \n Generate a SQL query to determine the ...,"SELECT \n vtype.type_name,\n SUM(s.sale_...",9
7,CREATE TABLE customer (\n customer_id INT P...,llama3.2:3b,[Write a SQL query to identify the top 3 custo...,"SELECT \n c.first_name, \n c.email,\n ...",8
8,CREATE TABLE warehouse (\n warehouse_id INT...,llama3.2:3b,[Write a SQL query to identify all products th...,"SELECT p.product_name, COUNT(od.order_detail_i...",6
9,CREATE TABLE products (\n product_id IN...,llama3.2:3b,[Write a SQL query to determine the total reve...,"SELECT s.name, p.name AS product_name, SUM(p.p...",8


In [41]:
hard_query_questions = []
for i in range(10):
    FILENAME = f"table_list{i+1}_exp.sql"
    ques=generate_questions(FILENAME,1,"HARD")
    hard_query_questions.append(ques)

In [42]:
final_dict2=[]

for model in model_list:
    for i in range(len(hard_query_questions)):
        FILENAME = f"table_list{i+1}_exp.sql"
        d = generate_sqlquery_and_rate(
            schema_file=FILENAME,
            model=model,
            question=hard_query_questions[i]
        )
        final_dict2.append(d)


In [43]:
df=pd.DataFrame(final_dict2)
df

,schema,model,question,sqlanswer,rating
0,\n CREATE TABLE supplier (\n supplie...,llama3.2:3b,[Design a query to identify the top 5 pharmaci...,"SELECT T1.pharmacy_name , SUM(T3.subtotal) AS...",8
1,CREATE TABLE store (\n store_id INT PRIMARY...,llama3.2:3b,[ Develop a SQL query to identify the top 5 ...,"SELECT T1.category_name, SUM(T2.subtotal) AS t...",8
2,CREATE TABLE user (\n user_id INT PRIMARY K...,llama3.2:3b,[Generate a SQL query to determine the average...,"SELECT \n c.category_name,\n AVG(o.total...",2
3,CREATE TABLE agent (\n agent_id INT PRIMARY...,llama3.2:3b,[ \n Generate a SQL query to identify all a...,"SELECT T1.first_name , T1.last_name , CAST(S...",3
4,CREATE TABLE supplier (\n supplier_id INT P...,llama3.2:3b,[Design a SQL query to identify the top 3 prod...,"SELECT T3.product_name, SUM(T2.quantity_sold *...",8
5,CREATE TABLE restaurant (\n restaurant_id I...,llama3.2:3b,[ Develop a SQL query to identify all restaur...,"SELECT \n r.name, r.location\n FROM \n rest...",2
6,CREATE TABLE vehicle_type (\n type_id INT P...,llama3.2:3b,[ \n Develop a SQL query to identify all cu...,"SELECT T1.first_name, T1.last_name, COUNT(DIST...",7
7,CREATE TABLE customer (\n customer_id INT P...,llama3.2:3b,[ \n Develop a SQL query to identify the to...,"SELECT t1.first_name, t1.email, COUNT(t3.ticke...",7
8,CREATE TABLE warehouse (\n warehouse_id INT...,llama3.2:3b,[ Develop a SQL query to identify all product...,"SELECT \np.product_name, \no.total_amount, \nw...",6
9,CREATE TABLE products (\n product_id IN...,llama3.2:3b,[Given the database schema containing informat...,"SELECT s.name, SUM(p.price * s.quantity) FROM...",7


In [44]:
df.to_csv('Result_4_LLM_Generated_Hard_Questions.csv')

#Questions created bt ChatGPT

In [ ]:
questions = [
    "List all drugs in the 'Painkillers' category with a stock below 100 units, along with their supplier names and the date when their inventory was last updated. Order results by oldest inventory update first.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including their category names and the store locations where they were sold. Order the results by total sales amount in descending order.",
    "Find the top 3 products with the highest average rating from user reviews, including their category names and total sales amounts. Only consider products that have been ordered at least 10 times. Order the results by average rating in descending order.",
    "List all agents who have facilitated sales of properties with a total sale price exceeding $500,000 in the last year. Include their full names, the number of properties sold, and the total commission earned. Order the results by total commission in descending order.",
    "Identify the top 5 products by total sales amount in the last quarter of 2024, including the name of the employee who made the sale and the total quantity sold for each product. Order the results by total sales amount in descending order.",
    "Retrieve the first name of customers who have given a 5-star rating to a restaurant and have also placed at least one order at that restaurant. For each such customer-restaurant pair, also retrieve the name of the restaurant and the total amount the customer has spent at that restaurant.",
    "Retrieve the first and last name of customers who have purchased a vehicle and also requested a service for that same vehicle, along with the model of the vehicle and the total cost of all services for that vehicle.",
    "How can you retrieve a list of all active subscriptions along with the customer's name, plan name, and the subscription's start and end dates",
    "How can you retrieve a report showing the total stock level of each product across all warehouses, including the product name and total quantity available?",
    "How can you generate a report showing the total sales revenue for each salesperson, including their name and the total amount sold?",
]

responses = []
ratings = []

import ollama

# model_list = ['qwen2.5:7b','gemma3:4b','phi3','phi3.5']
model_list = ['llama3.2:3b','gemma3:4b']

dict=[]

for model in model_list:
    
    for i in range(10):
        FILENAME = f"table_list{i+1}_exp.sql"

        response = ollama.chat(model=model, messages=[
        {
            'role': 'user',
            # 'content': get_prompt("What are the names and prices of all drugs in the 'Antibiotics' category?")
            'content': get_custom_prompt(
                question = questions[i],
                sql_file = FILENAME
            )
        },
        ])

        res = response['message']['content']
        

        responses.append(res)

        rating = get_ratings(
            schema = f"table_list{i+1}_exp.sql",
            question = questions[i],
            answer = res,
        )

        ratings.append(rating)

        d={
            'schema':read_file(FILENAME),
            'model':model,
            'question':questions[i],
            'sqlanswer':res,
            'rating':rating
        }

        dict.append(d)





In [50]:
df=pd.DataFrame(dict)
df.to_csv('Result_4_ChatGPT_Questions.csv')
df

,schema,model,question,sqlanswer,rating
0,\n CREATE TABLE supplier (\n supplie...,llama3.2:3b,List all drugs in the 'Painkillers' category w...,"SELECT DISTINCT D.drug_name, S.supplier_name, ...",3
1,CREATE TABLE store (\n store_id INT PRIMARY...,llama3.2:3b,Identify the top 5 products by total sales amo...,"SELECT T1.product_name, T2.category_name, T3.l...",3
2,CREATE TABLE user (\n user_id INT PRIMARY K...,llama3.2:3b,Find the top 3 products with the highest avera...,"SELECT T3.category_name, CAST(SUM(T2.quantity)...",3
3,CREATE TABLE agent (\n agent_id INT PRIMARY...,llama3.2:3b,List all agents who have facilitated sales of ...,"SELECT DISTINCT T1.first_name, T1.last_name, C...",6
4,CREATE TABLE supplier (\n supplier_id INT P...,llama3.2:3b,Identify the top 5 products by total sales amo...,"SELECT T1.product_name, T2.employee_name, SUM(...",4
5,CREATE TABLE restaurant (\n restaurant_id I...,llama3.2:3b,Retrieve the first name of customers who have ...,"SELECT T1.first_name, T3.name, SUM(T5.subtotal...",6
6,CREATE TABLE vehicle_type (\n type_id INT P...,llama3.2:3b,Retrieve the first and last name of customers ...,"SELECT T1.first_name, T1.last_name, T2.model, ...",6
7,CREATE TABLE customer (\n customer_id INT P...,llama3.2:3b,How can you retrieve a list of all active subs...,"SELECT \n c.first_name AS CustomerName,\n ...",9
8,CREATE TABLE warehouse (\n warehouse_id INT...,llama3.2:3b,How can you retrieve a report showing the tota...,"SELECT \n p.product_name,\n SUM(i.stock_...",10
9,CREATE TABLE products (\n product_id IN...,llama3.2:3b,How can you generate a report showing the tota...,"SELECT S.name, SUM(P.price * S.quantity) FROM ...",8


In [52]:
!ollama pull qwen2.5:3b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 5ee4f07cdb9b...  90% ▕██████████████  ▏ 1.7 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  91% ▕██████████████  ▏ 1.8 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  92% ▕██████████████  ▏ 1.8 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  93% ▕██████████████  ▏ 1.8 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  95% ▕███████████████ ▏ 1.8 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  96% ▕███████████████ ▏ 1.9 GB/1.9 GB                  pulling manifest 
pulling 5ee4f07cdb9b...  97% ▕███████████████ ▏ 1.9 GB/1.9 GB                

In [53]:
final_dict3=[]

model_list = ['llama3.2:3b','qwen2.5:3b']

for model in model_list:
    for i in range(len(hard_query_questions)):
        FILENAME = f"table_list{i+1}_exp.sql"
        d = generate_sqlquery_and_rate(
            schema_file=FILENAME,
            model=model,
            question=hard_query_questions[i]
        )
        final_dict3.append(d)

df=pd.DataFrame(final_dict3)
df.to_csv('Results.csv')
df


,schema,model,question,sqlanswer,rating
0,\n CREATE TABLE supplier (\n supplie...,llama3.2:3b,[Design a query to identify the top 5 pharmaci...,"SELECT T2.pharmacy_name, SUM(T3.subtotal) AS t...",8
1,CREATE TABLE store (\n store_id INT PRIMARY...,llama3.2:3b,[ Develop a SQL query to identify the top 5 ...,"SELECT T3.category_name, SUM(T4.subtotal) AS t...",7
2,CREATE TABLE user (\n user_id INT PRIMARY K...,llama3.2:3b,[Generate a SQL query to determine the average...,"SELECT T1.category_name, AVG(T2.total_amount) ...",2
3,CREATE TABLE agent (\n agent_id INT PRIMARY...,llama3.2:3b,[ \n Generate a SQL query to identify all a...,"SELECT T1.first_name, T1.last_name, AVG(T3.sa...",7
4,CREATE TABLE supplier (\n supplier_id INT P...,llama3.2:3b,[Design a SQL query to identify the top 3 prod...,"SELECT T1.product_name, SUM(T3.total_amount * ...",7
5,CREATE TABLE restaurant (\n restaurant_id I...,llama3.2:3b,[ Develop a SQL query to identify all restaur...,"SELECT r.name, COUNT(o.order_id) as total_nega...",3
6,CREATE TABLE vehicle_type (\n type_id INT P...,llama3.2:3b,[ \n Develop a SQL query to identify all cu...,"SELECT \n C.first_name,\n C.last_name,\n...",6
7,CREATE TABLE customer (\n customer_id INT P...,llama3.2:3b,[ \n Develop a SQL query to identify the to...,"SELECT \n c.first_name,\n c.email,\n ...",8
8,CREATE TABLE warehouse (\n warehouse_id INT...,llama3.2:3b,[ Develop a SQL query to identify all product...,"SELECT p.product_name, SUM(i.stock_level) AS ...",3
9,CREATE TABLE products (\n product_id IN...,llama3.2:3b,[Given the database schema containing informat...,"SELECT DISTINCT s.name , CAST(COALESCE(SUM(p...",7
